In [15]:
import sys
import os
import joblib
import pandas as pd
import torch

sys.path.insert(0, '..')
sys.path.insert(0, '../..')
sys.path.insert(0, '../../..')
sys.path.insert(0, '../../../..')
sys.path.insert(0, '../../../../..')
sys.path.insert(0, '../../../../../..')

from reimplemented_approaches.proactive_conformance_checking.data_prep_split_encode import DeviationLabeling, TrainTestSplit, Undersampling, PrefixDataset

In [16]:
# Data loading:
name_event_log = "Helpdesk"
path_event_log = "../../../../../../../../../data/data/helpdesk.csv"
path_process_model = "../../../../../../../data/process_models/Helpdesk.bpmn"

In [17]:
# Define attributes:
# Load the event log as pandas dataframe. Get all attributes in the log:
df = pd.read_csv(path_event_log)
print(df.columns)

Index(['CaseID', 'Activity', 'Resource', 'CompleteTimestamp', 'VariantIndex',
       'seriousness', 'customer', 'product', 'responsible_section',
       'seriousness_2', 'service_level', 'service_type', 'support_section',
       'workgroup'],
      dtype='object')


In [18]:
df

,CaseID,Activity,Resource,CompleteTimestamp,VariantIndex,seriousness,customer,product,responsible_section,seriousness_2,service_level,service_type,support_section,workgroup
0,Case 1,Assign seriousness,Value 1,2012/10/09 14:50:17.000,12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
1,Case 1,Take in charge ticket,Value 1,2012/10/09 14:51:01.000,12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1,Value 1
2,Case 1,Take in charge ticket,Value 2,2012/10/12 15:02:56.000,12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
3,Case 1,Resolve ticket,Value 1,2012/10/25 11:54:26.000,12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
4,Case 1,Closed,Value 3,2012/11/09 12:54:39.000,12,Value 1,Value 1,Value 1,Value 1,Value 1,Value 2,Value 1,Value 1,Value 1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
21343,Case 4579,Closed,Value 5,2010/09/02 10:11:00.000,1,Value 1,Value 71,Value 3,Value 1,Value 1,Value 3,Value 1,Value 1,Value 1
21344,Case 4580,Take in charge ticket,Value 6,2012/01/03 09:33:43.000,18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21345,Case 4580,Wait,Value 6,2012/01/10 15:30:11.000,18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1
21346,Case 4580,Resolve ticket,Value 6,2012/01/10 17:07:40.000,18,Value 1,Value 92,Value 3,Value 1,Value 2,Value 2,Value 2,Value 1,Value 1


In [19]:
# Data preparation and labelling
dl = DeviationLabeling(log_name=name_event_log,
                       path_event_log=path_event_log,
                       path_process_model=path_process_model,
                       label_strategy='separate')

# separate
dict_labeled_deviations, dict_encoders = dl.generate_individual_labels(trace_attr=['seriousness', 'customer', 'seriousness_2'], conf_runs=1)

  0%|          | 0/1 [00:00<?, ?it/s]

aligning log, completed variants ::   0%|          | 0/226 [00:00<?, ?it/s]

100%|██████████| 1/1 [00:05<00:00,  5.42s/it]


In [20]:
# df_labeled_deviations
for label, df in dict_labeled_deviations.items():
    print("Label: ", label)
    print("dataframe: ", df.shape)

Label:  ('>>', 'Assign seriousness')
dataframe:  (21348, 8)
Label:  ('Create SW anomaly', '>>')
dataframe:  (21348, 8)
Label:  ('Require upgrade', '>>')
dataframe:  (21348, 8)
Label:  ('Resolve ticket', '>>')
dataframe:  (21348, 8)
Label:  ('Take in charge ticket', '>>')
dataframe:  (21348, 8)
Label:  ('Wait', '>>')
dataframe:  (21348, 8)


In [21]:
act_ids = list(dict_encoders.values())[0]['activity_ids']
print("Activity ids: " ,act_ids)
print("\n")

res_ids = list(dict_encoders.values())[0]['resource_ids']
print("Reource ids: ", res_ids)
print("\n")

months = list(dict_encoders.values())[0]['month_ids']
print("Months: ", months)
print("\n")

deviations = list(dict_encoders.values())[0]['deviations']
print("Deviations: ", deviations)
print("\n")

Activity ids:  {'Assign seriousness': 1, 'Closed': 2, 'Create SW anomaly': 3, 'DUPLICATE': 4, 'INVALID': 5, 'Insert ticket': 6, 'RESOLVED': 7, 'Require upgrade': 8, 'Resolve SW anomaly': 9, 'Resolve ticket': 10, 'Schedule intervention': 11, 'Take in charge ticket': 12, 'VERIFIED': 13, 'Wait': 14}


Reource ids:  {'Value 1': 1, 'Value 10': 2, 'Value 11': 3, 'Value 12': 4, 'Value 13': 5, 'Value 14': 6, 'Value 15': 7, 'Value 16': 8, 'Value 17': 9, 'Value 18': 10, 'Value 19': 11, 'Value 2': 12, 'Value 20': 13, 'Value 21': 14, 'Value 22': 15, 'Value 3': 16, 'Value 4': 17, 'Value 5': 18, 'Value 6': 19, 'Value 7': 20, 'Value 8': 21, 'Value 9': 22}


Months:  {'10_2010': 1, '10_2011': 2, '10_2012': 3, '10_2013': 4, '11_2010': 5, '11_2011': 6, '11_2012': 7, '11_2013': 8, '12_2010': 9, '12_2011': 10, '12_2012': 11, '12_2013': 12, '1_2010': 13, '1_2011': 14, '1_2012': 15, '1_2013': 16, '1_2014': 17, '2_2010': 18, '2_2011': 19, '2_2012': 20, '2_2013': 21, '3_2010': 22, '3_2011': 23, '3_2012': 24, 

In [22]:
# Save prepared dataframe as .csv
label_dir = "./labelled_prefix_deviations"
os.makedirs(label_dir, exist_ok=True)

for label, df_label in dict_labeled_deviations.items():
    csv_path = os.path.join(label_dir, f"{label}_prefix_deviations.csv")
    df_label.to_csv(csv_path, index=False)

In [23]:
# Train and test split
tts = TrainTestSplit(df_labled_deviations=dict_labeled_deviations,
                     label_strategy="separate")
train_dict, val_dict, test_dict = tts.data_split(val_frac=0.2)

In [24]:
# df_labeled_deviations
for label, df in train_dict.items():
    print("Label: ", label)
    print("dataframe: ", df.shape)

Label:  ('>>', 'Assign seriousness')
dataframe:  (11386, 8)
Label:  ('Create SW anomaly', '>>')
dataframe:  (11386, 8)
Label:  ('Require upgrade', '>>')
dataframe:  (11386, 8)
Label:  ('Resolve ticket', '>>')
dataframe:  (11386, 8)
Label:  ('Take in charge ticket', '>>')
dataframe:  (11386, 8)
Label:  ('Wait', '>>')
dataframe:  (11386, 8)


In [26]:
u = Undersampling(train_data=train_dict,
                  list_dynamic_cols=['activities', 'resources', 'months'],
                  label_strategy='separate')
train_dict, y_no_true_class = u.one_sided_selection_undersampling()

In [27]:
# df_labeled_deviations
for label, df in train_dict.items():
    print("Label: ", label)
    print("dataframe: ", df.shape)

Label:  ('>>', 'Assign seriousness')
dataframe:  (10816, 8)
Label:  ('Create SW anomaly', '>>')
dataframe:  (11053, 8)
Label:  ('Require upgrade', '>>')
dataframe:  (11304, 8)
Label:  ('Resolve ticket', '>>')
dataframe:  (11187, 8)
Label:  ('Take in charge ticket', '>>')
dataframe:  (10999, 8)
Label:  ('Wait', '>>')
dataframe:  (11110, 8)


In [28]:
y_columns = {label: [c for c in df_label.columns if c.startswith("y_")] for label, df_label in train_dict.items()}

for label in y_columns:
    dict_encoders[label]['deviations'] = y_columns[label]
joblib.dump(dict_encoders, "encoders.pkl")

['encoders.pkl']

In [29]:
# Tensor encoding and saving — quick literal fix
# device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
device = torch.device("cpu")
dataset_prep = PrefixDataset(df_train=train_dict,
                             df_val=val_dict,
                             df_test=test_dict,
                             activity_col='activities',
                             resource_col='resources',
                             month_col='months',
                             trace_cols=['trace_attr_seriousness', 'trace_attr_customer', 'trace_attr_seriousness_2'],
                             y_cols=y_columns,
                             label_strategy="separate")

train_set_dict, val_set_dict, test_set_dict = dataset_prep.tensor_datset_encoding(device=device)
dataset_prep.save_datasets(train_dataset=train_set_dict,
                           val_dataset=val_set_dict,
                           test_dataset=test_set_dict,
                           save_path=".")

('./train_set.pkl', './val_set.pkl', './test_set.pkl')

In [30]:
train_set_dict, val_set_dict, test_set_dict = dataset_prep.load_datasets(".")
for label in train_set_dict:
    print(label, train_set_dict[label].tensors[0].shape, test_set_dict[label].tensors[0].shape)

('>>', 'Assign seriousness') torch.Size([10816, 15]) torch.Size([7116, 15])
('Create SW anomaly', '>>') torch.Size([11053, 15]) torch.Size([7116, 15])
('Require upgrade', '>>') torch.Size([11304, 15]) torch.Size([7116, 15])
('Resolve ticket', '>>') torch.Size([11187, 15]) torch.Size([7116, 15])
('Take in charge ticket', '>>') torch.Size([10999, 15]) torch.Size([7116, 15])
('Wait', '>>') torch.Size([11110, 15]) torch.Size([7116, 15])
